## Step1: Specialization Vector

In [ ]:
## Create a one-hot vector for each paper indicating whether or not each author wrote it
## This one-hot vector is used for creating specialization vectors of authors

In [1]:
import pickle
import pandas as pd
import numpy as np

In [2]:
# Read authids in each section of the guideline
authid_12 = np.load('/disks/qnap3/users/21-tomokiyo/masters_thesis/data/guideline_paper/member_data/2021/A1.2_who_guideline_development_group.npy')
authid_13 = np.load('/disks/qnap3/users/21-tomokiyo/masters_thesis/data/guideline_paper/member_data/A1.3_who_systematic_review_team_revise.npy')
authid_14 = np.load('/disks/qnap3/users/21-tomokiyo/masters_thesis/data/guideline_paper/member_data/2021/A1.4_who_external_methodologists.npy')
authid_15 = np.load('/disks/qnap3/users/21-tomokiyo/masters_thesis/data/guideline_paper/member_data/A1.5_who_external_review_team_revise.npy')

boradmember_authid = list(authid_12)+list(authid_13)+list(authid_14)+list(authid_15)

In [3]:
additional_embedding_df = pd.read_pickle('/disks/qnap3/users/21-tomokiyo/masters_thesis/data/guideline_paper/all_guideline_matching/additional_embedding_2011-2015.pickle')
additonal_embedding_eid = additional_embedding_df['eid'].values
additonal_embedding_eid = additonal_embedding_eid.astype(int)
additonal_embedding_eid

array([84875442265, 84861568429, 85026817599])

In [4]:
authid_list = [35780677200 if x == 7006806977 else 57206422848 if x == 54791745100  else 57219144066 if x == 55588756600  else 57219144066 if x == 57191727610  else 57219144066 if x == 14037485900 else x for x in boradmember_authid]

In [5]:
embedding_eid_ls = np.load('/disks/qnap3/users/21-tomokiyo/masters_thesis/data/guideline_paper/all_guideline_matching/embedding_eid_ls.npy')
embedding_eid_ls1 = list(embedding_eid_ls) + list(additonal_embedding_eid)

In [6]:
author_paperlist_df = pd.read_pickle('/disks/qnap3/shared/scopus-24/data/author/author_paperlist.pickle')
author_paperlist_df = pd.DataFrame(author_paperlist_df)

In [7]:
author_paperlist_df.head()

,author_paperlist
authids,
1,"[14881442, 14881430, 14881456, 14881394, 14881..."
2,"[58149416836, 33744754941, 58149416551, 157474..."
4,"[17858319, 19206542, 19982084, 24229822, 84886..."
6,"[17644442848, 16023755, 84916826692, 851127286..."
8,"[77956986345, 15661038, 17414010, 19533239, 21..."


In [8]:
author_paperlist_df_explode = author_paperlist_df.explode('author_paperlist')

In [9]:
author_paperlist_df_explode.rename(columns={'author_paperlist':'eid'},inplace=True)

In [10]:
author_paperlist_df_explode.reset_index(inplace=True)

In [12]:
author_paperlist_df_explode1_1 = author_paperlist_df_explode[author_paperlist_df_explode['eid'].isin(embedding_eid_ls1)]

In [13]:
authid_list = [35780677200 if x == 7006806977 else 57206422848 if x == 54791745100  else 57219144066 if x == 55588756600  else 57219144066 if x == 57191727610  else 57219144066 if x == 14037485900 else x for x in boradmember_authid]
authid_list = list(set(authid_list))
len(authid_list)

108

In [14]:
author_paperlist_df_explode1_1.replace({7006806977:35780677200, 54791745100:57206422848, 55588756600:57219144066, 57191727610:57219144066, 14037485900:57219144066},inplace=True)

/tmp/ipykernel_942534/2342031312.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  author_paperlist_df_explode1_1.replace({7006806977:35780677200, 54791745100:57206422848, 55588756600:57219144066, 57191727610:57219144066, 14037485900:57219144066},inplace=True)
/tmp/ipykernel_942534/2342031312.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  author_paperlist_df_explode1_1.replace({7006806977:35780677200, 54791745100:57206422848, 55588756600:57219144066, 57191727610:57219144066, 14037485900:57219144066},inplace=True)


In [163]:
# author_paperlist_df_explode1.head(50).to_csv('create_one_hot_for_gpt.csv')

In [16]:
author_ls = author_paperlist_df_explode1_1.authids.unique()

In [17]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix

df = author_paperlist_df_explode1_1
unique_authids = df["authids"].unique()
unique_eids = embedding_eid_ls1

authid_to_idx = {authid: idx for idx, authid in enumerate(unique_authids)}
eid_to_idx = {eid: idx for idx, eid in enumerate(unique_eids)}

row_indices = df["authids"].map(authid_to_idx).values
col_indices = df["eid"].map(eid_to_idx).values

# sparse_matrix: one-hot encoding that represents the presence of each authid in each eid
sparse_matrix = csr_matrix((np.ones(len(df), dtype=np.int8), (row_indices, col_indices)), 
                           shape=(len(unique_authids), len(unique_eids)))

# Save file
# output_file = "/disks/qnap3/users/21-tomokiyo/masters_thesis/data/guideline_paper/all_guideline_matching/sparse_one_hot_matrix.npz"
#np.savez_compressed(output_file, data=sparse_matrix.data, indices=sparse_matrix.indices, indptr=sparse_matrix.indptr, shape=sparse_matrix.shape)
#print(f"Sparse Matrix Shape: {sparse_matrix.shape}")
#print(f"Non-zero Elements: {sparse_matrix.nnz}")
#print(f"Saved File Path: {output_file}")
